# Setup

## Dependencies

In [ ]:
import pandas as pd
import os.path
from os import path
import requests
import json

## Files

In [ ]:
qc_file = 'Quality Control File 073020.xlsx'
qc_out_file = 'Quality Control File 073020 with Leaders.xlsx'

## Functions


Function for quickly obtaining column names:

In [ ]:
def get_cols(df, substring):
    return list(df.columns[df.columns.str.contains(substring)])

# HLA-B Leader 

In [ ]:
qc_df = pd.read_excel(qc_file)
qc_df

# REST Endpoints
Let's go through each REST call useful for leader annotation and process a row from our HLA dataset as inputs.

## HLA-B Leader Allotype Classification

### Row
This is the example row for the function set up. Feel free to change the indexing to a different row:

In [ ]:
row = qc_df.iloc[1]
row

### Parameters

The parameter for this first GET endpoint is simply a B allele. For example:

In [ ]:
param = row['HLA_B 1']
param

### REST Call

This function calls the local server to classify the leader type of an HLA-B allele.

In [ ]:
def classify_leader_allotype(hla_b):
    if 'B*' not in hla_b:
        hla_b = 'B*' + hla_b
    url = "http://localhost:5010/b-leader-classification/allotype"
    headers = {'content-type': 'application/json'}
    params = {'allele' : hla_b}
    try:
        response = requests.post(url,
                                headers=headers,
                                data=json.dumps(params))
        results = json.loads(response.content)
        return results
    except Exception as e:
        print(e)
        raise e
classify_leader_allotype(param)[0]['common_leader']

In [ ]:
for index, row in qc_df.iterrows():
    if index % 10 == 0:
        print(index)
    for i in range(2):
        results = classify_leader_allotype(row['HLA_B %s' % (i+1)])
        leader_type = results[0]['common_leader']
        if leader_type not in ['M', 'T']:
            print(index, leader_type, results)
        qc_df.loc[index, 'Leader %s' % (i+1)] = leader_type
qc_df.to_xlsx(qc_out_file)

In [ ]:
qc_df.to_excel(qc_out_file)

# Iterating through the dataset
With all these functions set up, we can iterate through a list of recipient-donor typings to get our leader match statuses.

# Done!
This is the end of the demonstration. Thank you! Please use this notebook and any of the above chunks for your own use.